In [ ]:
!pip install pydub
!pip install google-cloud-speech
!pip install srt

In [114]:
import os
import srt
import datetime

In [15]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "amazing-aleph-351511-a12e8ed121dc.json"

In [153]:
from pydub import AudioSegment
from pydub.utils import make_chunks
from google.cloud import speech

In [319]:
root='dir/'
file='video_file'

In [323]:
path = root+'59S_16K'

In [ ]:
video = AudioSegment.from_file(f"{root}{file}.mp4", "mp4")
audio = video.set_channels(1)

In [301]:
audio = audio.set_frame_rate(16000)

In [302]:
chunk_length_ms = 59 * 1000 # pydub calculates in millisec
chunks = make_chunks(audio, chunk_length_ms)

In [ ]:
for i, chunk in enumerate(chunks):
    chunk_name = path+"/chunk{0}.wav".format(i)
    print("exporting", chunk_name)
    chunk.export(chunk_name, format="wav")

## Recognize Google

In [304]:
def transcribe(audio_file, enable_automatic_punctuation=False):
    client = speech.SpeechClient()
    
    # Load and convert audio file
    audio_segment = AudioSegment.from_wav(audio_file)
    audio_segment = audio_segment.set_channels(1)
    content = audio_segment.export(format="wav").read()
    audio = speech.RecognitionAudio(content=content)
    
    # Configure API request
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code="en-US",
        model= "video",
        enable_word_time_offsets=True,
        enable_automatic_punctuation=enable_automatic_punctuation
    )
    
    # Send request to API and get response
    response = client.recognize(config=config, audio=audio)
    
    return response

In [305]:
response_p0 = transcribe(path+"/chunk0.wav", enable_automatic_punctuation=True)

In [306]:
response_p1 = transcribe(path+"/chunk1.wav", enable_automatic_punctuation=True)

In [307]:
response_p2 = transcribe(path+"/chunk2.wav", enable_automatic_punctuation=True)

In [308]:
response_p3 = transcribe(path+"/chunk3.wav", enable_automatic_punctuation=True)

In [ ]:
response_p2

In [322]:
# this code is modified to include punctuation and offsets. 
# I do not remember the original code to cite.
def subtitle_generation_wpunc(speech_to_text_response, offset_seconds=0, offset_micros=0):
    transcriptions = []
    index = 0
 
    for result in speech_to_text_response.results:
        try:
            if result.alternatives[0].words[0].start_time.seconds:
                start_sec = result.alternatives[0].words[0].start_time.seconds 
                start_microsec = result.alternatives[0].words[0].start_time.microseconds
            else:
                start_sec = 0
                start_microsec = 0 

            last_word_end_sec = result.alternatives[0].words[-1].end_time.seconds
            last_word_end_microsec = result.alternatives[0].words[-1].end_time.microseconds
            
            transcript = result.alternatives[0].words[0].word
            
            index += 1 

            for i in range(len(result.alternatives[0].words) - 1):
                try:
                    word = result.alternatives[0].words[i + 1].word
                    word_start_sec = result.alternatives[0].words[i + 1].start_time.seconds
                    word_start_microsec = result.alternatives[0].words[i + 1].start_time.microseconds
                    word_end_sec = result.alternatives[0].words[i + 1].end_time.seconds
                    word_end_microsec = result.alternatives[0].words[i + 1].end_time.microseconds
                        
                    transcript = transcript + " " + word
                    
                    if word[-1] in ['.','?','!']:
                        
                        previous_word_end_sec = result.alternatives[0].words[i].end_time.seconds
                        previous_word_end_microsec = result.alternatives[0].words[i].end_time.microseconds
                        
                        td_start = datetime.timedelta(0, start_sec+offset_seconds, start_microsec+offset_micros)
                        td_end = datetime.timedelta(0, previous_word_end_sec+offset_seconds, previous_word_end_microsec+offset_micros)
                        
                        transcriptions.append(srt.Subtitle(index, td_start, td_end, transcript))
                        
                        #print(offset_seconds, offset_micros, td_start, td_end)
                        
                        start_sec = word_start_sec
                        start_microsec = word_start_microsec
                        
                        # reset
                        transcript = ''
                        
                        index += 1
                except IndexError:
                    pass

            td_start = datetime.timedelta(0, start_sec+offset_seconds, offset_micros+start_microsec)
            td_end = datetime.timedelta(0, last_word_end_sec+offset_seconds, offset_micros+last_word_end_microsec)
            transcriptions.append(srt.Subtitle(index,td_start , td_end , transcript))
            
            # print(offset_seconds, offset_micros, td_start, td_end)
            
            index += 1
        except IndexError:
            pass
    
    # turn transcription list into subtitles
    subtitles = srt.compose(transcriptions)
    return subtitles

In [311]:
transacript = subtitle_generation_wpunc(response_p0, offset_seconds=0)

In [312]:
transacript += subtitle_generation_wpunc(response_p1, offset_seconds=chunk_length_ms/1000)

In [313]:
transacript += subtitle_generation_wpunc(response_p2, offset_seconds=2*chunk_length_ms/1000)

In [314]:
transacript += subtitle_generation_wpunc(response_p3, offset_seconds=3*chunk_length_ms/1000)

In [ ]:
transacript

In [320]:
with open(root+"/"+file+".srt", "w") as f:
    f.write(transacript)